In [1]:
from common.arguments import parse_args
from common.camera import *
from common.visualization import *
from common.utils import *
from common.generators import ChunkedGenerator, UnchunkedGenerator
from common.h36m_dataset import Human36mDataset, preprocess_Human36m
import matplotlib
%matplotlib inline

In [2]:
args = parse_args()
args.subjects_unlabeled='S5,S6'

In [3]:
dataset_path = "./data/data_3d_h36m.npz"
dataset_path_2d = './data/data_2d_h36m_cpn_ft_h36m_dbb.npz'

In [4]:
dataset, keypoints, kps_left, kps_right, joints_left, joints_right = preprocess_Human36m(dataset_path, dataset_path_2d)

Loading dataset...
Preparing data...
Loading 2D detections...


In [5]:
subjects_train = args.subjects_train.split(',')
subjects_semi = [] if not args.subjects_unlabeled else args.subjects_unlabeled.split(',')
subjects_test = args.subjects_test.split(',')

In [8]:
subjects_test

['S9', 'S11']

In [9]:
action_filter = None if args.actions == '*' else args.actions.split(',')

In [7]:
cameras_valid, poses_valid, poses_valid_2d = fetch(dataset, keypoints, subjects_test, args.stride, action_filter)

In [30]:
cameras_semi, _, poses_semi_2d = fetch(dataset, keypoints, subjects_semi, args.stride, action_filter, parse_3d_poses=False)

In [8]:
cameras_train, poses_train, poses_train_2d = fetch(dataset, keypoints, subjects_train, args.stride, action_filter, subset=args.subset)

In [9]:
train_generator = ChunkedGenerator(args.batch_size//args.stride, 
                                   cameras_train, 
                                   poses_train, 
                                   poses_train_2d, 
                                   args.stride,
                                   pad=13, 
                                   causal_shift=0, 
                                   shuffle=True, 
                                   augment=args.data_augmentation,
                                   kps_left=kps_left, 
                                   kps_right=kps_right, 
                                   joints_left=joints_left, 
                                   joints_right=joints_right)

In [17]:
train_generator_eval = UnchunkedGenerator(cameras_train, 
                                          poses_train, 
                                          poses_train_2d,
                                          pad=13, 
                                          causal_shift=0, 
                                          augment=False)

In [32]:
semi_generator = ChunkedGenerator(args.batch_size//args.stride, 
                                  cameras_semi, 
                                  None, 
                                  poses_semi_2d, 
                                  args.stride,
                                  pad=27, 
                                  causal_shift=0, 
                                  shuffle=True,
                                  random_seed=4321, 
                                  augment=args.data_augmentation,
                                  kps_left=kps_left, 
                                  kps_right=kps_right, 
                                  joints_left=joints_left, 
                                  joints_right=joints_right,
                                  endless=True)

In [22]:
for a, batch_3d, batch_2d in train_generator.next_epoch():
    break

In [24]:
a.shape

(1024, 9)

In [25]:
batch_3d.shape

(1024, 1, 17, 3)

In [26]:
batch_2d.shape

(1024, 27, 17, 2)